In [58]:
import pandas as pd
import numpy as np
import os
from google.colab import drive
import pandas as pd
import ast

In [59]:
drive.mount('/content/gdrive')

path = "/content/gdrive/My Drive/Capstone Code"
os.chdir(path)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [60]:
bio = pd.read_csv('./data/uch_anon_hrher2.csv')

In [61]:
bio.head()

,slide_anon_name,patient_anon,ER_List,PR_List,HER2_IHC
0,UCH_BRCA_1.svs,1,['90'],['50'],Negative (0)
1,UCH_BRCA_2.svs,2,['0'],['0'],Negative (0 - 1+)
2,UCH_BRCA_3.svs,3,['0'],['0'],Equivocal (2+)
3,UCH_BRCA_4.svs,4,[],['0'],Positive (3+)
4,UCH_BRCA_5.svs,5,"['95', '100']","['100', '40', '5']",Equivocal (2+)


In [62]:
df_er = bio[['slide_anon_name', 'patient_anon', 'ER_List']]
df_pr = bio[['slide_anon_name', 'patient_anon', 'PR_List']]
df_her2_ihc = bio[['slide_anon_name', 'patient_anon', 'HER2_IHC']]

print('df_er head:')
display(df_er.head())

print('\ndf_pr head:')
display(df_pr.head())

print('\ndf_her2_ihc head:')
display(df_her2_ihc.head())

df_er head:


,slide_anon_name,patient_anon,ER_List
0,UCH_BRCA_1.svs,1,['90']
1,UCH_BRCA_2.svs,2,['0']
2,UCH_BRCA_3.svs,3,['0']
3,UCH_BRCA_4.svs,4,[]
4,UCH_BRCA_5.svs,5,"['95', '100']"



df_pr head:


,slide_anon_name,patient_anon,PR_List
0,UCH_BRCA_1.svs,1,['50']
1,UCH_BRCA_2.svs,2,['0']
2,UCH_BRCA_3.svs,3,['0']
3,UCH_BRCA_4.svs,4,['0']
4,UCH_BRCA_5.svs,5,"['100', '40', '5']"



df_her2_ihc head:


,slide_anon_name,patient_anon,HER2_IHC
0,UCH_BRCA_1.svs,1,Negative (0)
1,UCH_BRCA_2.svs,2,Negative (0 - 1+)
2,UCH_BRCA_3.svs,3,Equivocal (2+)
3,UCH_BRCA_4.svs,4,Positive (3+)
4,UCH_BRCA_5.svs,5,Equivocal (2+)


## Functions for checking

In [87]:
def check_condition(er_list_str):## check the "0" at the begin of the char
    # Convert the string to an actual list
    try:
        er_list = ast.literal_eval(er_list_str)
    except (ValueError, SyntaxError):
        return False  # Handle invalid format gracefully

    if not er_list:

        return False
    # Check if the list starts with '0' and contains another value greater than 0
    if er_list[0] == '0' and any(int(x) > 0 for x in er_list[1:] if x.isdigit()):
        print(er_list)
        return True
    return False


In [90]:
# Function to check the condition for string values in ER_List
def check_zero_condition(er_list_str):## if the str contant more then one value, and check whether there is "0" in the list
    # Convert the string to an actual list
    try:
        er_list = ast.literal_eval(er_list_str)
    except (ValueError, SyntaxError):
        return False  # Handle invalid format gracefully

    # Check if the list contains more than one value
    if len(er_list) > 1:
        # Count if '0' is present in the list
        if '0' in er_list:
            print(er_list)
            return True
    return False


## Patient Checking

In [64]:
bio['patient_anon'].value_counts()

,count
patient_anon,
318,15
339,13
398,12
391,12
332,12
...,...
12,1
11,1
8,1


## Cleaning process for HER2 dataset: df_her2_ihc

In [65]:
allowed_values = ['Negative (0)', 'Negative (0 - 1+)', 'Equivocal (2+)', 'Positive (3+)']
df_her2_ihc = df_her2_ihc[df_her2_ihc['HER2_IHC'].isin(allowed_values)]
df_her2_ihc['HER2_IHC'] = df_her2_ihc['HER2_IHC'].str.replace(r'\s*\(.*\)', '', regex=True)
display(df_her2_ihc.head())

,slide_anon_name,patient_anon,HER2_IHC
0,UCH_BRCA_1.svs,1,Negative
1,UCH_BRCA_2.svs,2,Negative
2,UCH_BRCA_3.svs,3,Equivocal
3,UCH_BRCA_4.svs,4,Positive
4,UCH_BRCA_5.svs,5,Equivocal


In [66]:
df_her2_ihc["HER2_IHC"].value_counts()

,count
HER2_IHC,
Negative,462
Equivocal,228
Positive,190


## Cleaning process for ER dataset: df_er

In [73]:
len(df_er['ER_List'])

1175

In [67]:
df_er['ER_List'].value_counts()

,count
ER_List,
['90'],245
['0'],194
['95'],75
"['90', '95']",55
"['0', '1']",54
...,...
"['0', '3']",1
"['80', '8']",1
"['98', '8']",1


In [83]:
len(df_er[df_er['ER_List'] == "['5-10', '0']"])

8

In [85]:
len(df_er[df_er['ER_List'] == "[]"])

16

In [69]:
count = df_er['ER_List'].apply(check_condition).sum()

print(f'Count of rows with 0 followed by a number greater than 0: {count}')

['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '5']
['0', '5']
['0', '5']
['0', '5']
['0', '5']
['0', '5']
['0', '5']
['0', '5']
['0', '5']
['0', '80']
['0', '80']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '75']
['0', '75']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '50']
['0', '50']
['0', '50']
['0', '50']
['0', '50']
['0', '50']
['0', '50']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '60']
['0', '60']
['0', '60']
['0', '60']
['0', '60']
['0', '60']
['0', '75']
['0', '1', '5']
['0', '1']
['0', '1']
['0', '1']
['0', '50']
['0', '1']
['0', '50']
['0', '1']
['0', '1']
['0', '1']
['0', '10']
['0', '3']
['0', '1']
['0', '1']
['0', '1']
Count of rows with 0 followe

In [91]:
# Apply the function to the ER_List column and count the matches
count = df_er['ER_List'].apply(check_zero_condition).sum()

print(f'Count of rows with more than one value and containing 0: {count}')

['30', '0']
['90', '0']
['5', '0', '1']
['0', '1']
['95', '0', '50']
['40', '0']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['88', '95', '0']
['0', '5']
['0', '5']
['0', '5']
['0', '5']
['0', '5']
['0', '5']
['0', '5']
['0', '5']
['0', '5']
['70', '0']
['70', '0']
['70', '0']
['70', '0']
['70', '0']
['70', '0']
['0', '80']
['0', '80']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '75']
['0', '75']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['30', '0']
['30', '0']
['0', '50']
['0', '50']
['0', '50']
['0', '50']
['0', '50']
['0', '50']
['0', '50']
['0', '1']
['0', '1']
['0', '1']
['5-10', '0']
['5-10', '0']
['5-10', '0']
['5-10', '0']
['5-10', '0']
['5-10', '0']
['5-10', '0']
['5-10', '0']
['0', '1']
['0', '1']
['0', '1']


In [92]:
1175-16-194-116

849

## Cleaning process for PR dataset: df_pr

In [70]:
df_pr['PR_List'].value_counts()

,count
PR_List,
['0'],463
['90'],89
['1'],42
['80'],39
"['0', '1']",31
...,...
"['90', '50']",1
"['90', '95', '60']",1
"['0', '60', '80']",1


In [71]:
len(df_pr[df_pr['PR_List'] == "[]"])

9

In [72]:
count = df_pr['PR_List'].apply(check_condition).sum()

print(f'Count of rows with 0 followed by a number greater than 0: {count}')

['0', '50']
['0', '5']
['0', '10']
['0', '5']
['0', '1']
['0', '10']
['0', '1']
['0', '1']
['0', '5']
['0', '10']
['0', '10']
['0', '25']
['0', '50']
['0', '50']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '80']
['0', '80']
['0', '80']
['0', '80']
['0', '80']
['0', '80']
['0', '80']
['0', '80']
['0', '80']
['0', '3']
['0', '3']
['0', '3']
['0', '3']
['0', '3']
['0', '10']
['0', '10']
['0', '1']
['0', '1']
['0', '10', '50']
['0', '15']
['0', '10']
['0', '50']
['0', '80']
['0', '5']
['0', '1']
['0', '5']
['0', '5']
['0', '1']
['0', '3', '1']
['0', '3', '1']
['0', '60', '80']
['0', '1']
Count of rows with 0 followed by a number greater than 0: 69


In [93]:
# Apply the function to the ER_List column and count the matches
count = df_pr['PR_List'].apply(check_zero_condition).sum()

print(f'Count of rows with more than one value and containing 0: {count}')

['0', '50']
['0', '5']
['0', '10']
['90', '0']
['0', '5']
['20', '0']
['20', '0']
['20', '0']
['0', '1']
['0', '10']
['0', '1']
['0', '1']
['25', '80', '0']
['80', '0']
['51-100', '0']
['51-100', '0']
['0', '5']
['20', '0']
['0', '10']
['0', '10']
['0', '25']
['0', '50']
['0', '10-20']
['30', '0']
['30', '0']
['30', '0']
['0', '50']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['90', '0']
['90', '0']
['90', '0']
['90', '0']
['90', '0']
['90', '0']
['90', '0']
['90', '0']
['90', '0']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['0', '1']
['20', '0']
['0', '80']
['0', '80']
['0', '80']
['0', '80']
['0', '80']
['0', '80']
['0', '80']
['0', '80']
['0', '80']
['0', '3']
['0', '3']
['0', '3']
['0', '3']
['0', '3']
['20', '0']
['0', '10']
['80', '0']
['80', '0']
['0', '10']
['0', '1']
['30', '0']
['25', '80', '0']
['0', '1']
['80', '0']
['0',

In [95]:
len(df_pr['PR_List']) - 9 - 463 - 115

588